# Scatter Plots

## Read in Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
# file_names = !ls out*.csv
# df_list = list(map(lambda f: pd.read_csv(f), file_names))
# df = pd.concat(df_list).reset_index(drop=True)
# df.sort_values(by=['lpobs'], ascending = False, inplace = True)

# # #remove outliers
# # df = df.loc[:, df.columns != 'word']
# # df = df[(np.abs(stats.zscore(df)) < 2).all(axis=1)]

# num = np.arange(0, len(df.index))
# df['index'] = num.tolist()
# N_DAYS = 100
# # df['mean'] = df['mu']+df['a']
# df.head(5)
# # df.to_csv('mod7df.csv')

In [ ]:
# df.shape
# df.to_csv('out999.csv',index = None)

In [8]:
df = pd.read_csv('out999.csv')
print(df.shape)
df.head()


FileNotFoundError: [Errno 2] File b'out999.csv' does not exist: b'out999.csv'

In [ ]:
df['maxr'].describe()
# df.loc[df['word'] == 'many']


## Show plot of Correlation

In [ ]:
print(len(df['maxr']))
xval = [1,1.01,1.02,1.05, 1.08,1.1,1.15,1.2,1.25,1.3]
yval = []
for x in xval:
    yval.append(df[df['maxr'] < x]['maxr'].count()/len(df['word']))

plt.plot(xval,yval, linestyle = '--')
plt.ylabel('% of data')
plt.xlabel('maxr val')
plt.show()

In [ ]:
## Show Plot of Rhat
print(len(df[df['maxr'] < 1.1])/len(df['maxr']))
df['maxr'].describe()
dfr = df[np.abs(df.maxr-df.maxr.mean())<=(2*df.maxr.std())]
plt.hist(dfr['maxr'], bins = 30)
plt.show()

In [ ]:
#Graphs index vs rhat in seaborn with correlation

# c = stats.kendalltau(df['maxr'], df['index'])[0]
c2 = df['index'].corr(df['maxr'])
import seaborn as sns
ax =sns.regplot(x=dfr['index'], y=dfr['maxr'],
                scatter_kws={"color": "black", "alpha":0.2}, 
                line_kws={"color": "blue"})

plt.title("Frequency Index vs Max Rhat \n Correlation = %.2f"%(c2), size = 16)
plt.xlabel("Frequency Rank", size = 14)
plt.ylabel("Max Rhat", size = 14)
plt.show(ax)

## Correlation (Kendall-tau)

In [ ]:
x = 'mu'
variables = ['alpha', 'sd', 'a', 'drift','theta', 'lpobs', 'maxr', 'index']
corr = {}
for var in variables:
    print("Corr: mu vs %s" %var)
    print(df[x].corr(df[var]))
    corr[var] = df[x].corr(df[var])

## Scatter Plots

# Word Specific Plots

In [7]:
df.loc[df['word']=='obama']

NameError: name 'df' is not defined

In [11]:
dftrump = pd.read_csv('lpout252.csv')
dftrump.head()

,day,lpobs,mu,low,high
0,0,-10.761344,-10.648427,-10.809986,-10.466258
1,1,-9.933443,-9.937457,-9.981711,-9.888887
2,2,-10.058613,-10.030197,-10.088783,-9.967884
3,3,-8.694873,-8.693549,-8.721677,-8.664824
4,4,-8.146013,-8.150288,-8.168051,-8.133105


In [ ]:
def skew_norm_pdf(x,e=0,w=1,a=0):
    # adapated from:
    # http://stackoverflow.com/questions/5884768/skew-normal-distribution-in-scipy
    t = (x-e) / w
    return 2.0 * w * stats.norm.pdf(t) * stats.norm.cdf(a*t)

## Plot Trump

In [ ]:
dfobs = pd.read_csv('Output52.txt', sep = ' ', names = ['day','word','cnt','z'])
dfobs['lpobs'] = np.log(dfobs['cnt']/dfobs['z'])

dfobs= dfobs.sort_values('day', ascending = True)
#Create Bounds
n=63
mu = df.iloc[n,:]['mu']
sd = df.iloc[n,:]['sd']
alpha = df.iloc[n,:]['alpha']
drift = df.iloc[n,:]['drift']

x0,x1,y0,y1 = dfobs['lpobs'].min()-0.5, dfobs['lpobs'].max()+0.5, dfobs['day'].max(), dfobs['day'].min()
x = np.linspace(x0,x1,100) 
p = skew_norm_pdf(x,mu,sd,alpha)

# plot it
from matplotlib import gridspec
gs = gridspec.GridSpec(3, 3)

fig = plt.figure(figsize=(6, 8)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 5]) 
ax0 = plt.subplot(gs[0])
ax0.plot(x, p)
ax0.text(-12,0.15,'mu = %.2f'%mu)
ax0.text(-12,0.1,'alpha = %.2f'%alpha)
ax0.text(-12,0.05,'sd = %.2f'%sd)
ax0.set_title("Lp Distribution for: 'Terrorist'")
ax0.grid(False)
plt.setp(ax0.get_xticklabels(), visible=False)

ax1 = plt.subplot(gs[1], sharex = ax0)
line1, = ax1.plot(dfobs['lpobs'],dfobs['day'], alpha = 0.7, linewidth = 1.4, label = 'lp observed')
ax1.axis([x0,x1,y0,y1])
ax1.set_ylabel("Day")
ax1.set_xlabel('Log Probability')
ax1.grid(linewidth = 0.5)
#####horizontal line
xs = np.linspace(dfobs['lpobs'].min()-1, dfobs['lpobs'].max()+1,200)
horiz_line_data = np.array([231 for i in range(len(xs))])
#ax1.plot(xs, horiz_line_data, 'k--', linewidth = 0.7) 
#ax1.text(-12,250,'Nice Terror Attack', size = 11, fontweight = 'bold')
#horiz_line_data_2 = np.array([290 for i in range(len(xs))])
#ax1.plot(xs, horiz_line_data_2, 'k--', linewidth = 0.7) 
#ax1.text(-12,309,'Pipe Bomb in New York', size = 11, fontweight = 'bold')

# ax2 = ax1.twinx()
# line2, = ax2.plot(dftrumplp['mu'], dftrumplp['day'], '-.', alpha = 0.7, c = 'r', linewidth = 0.4, label = 'lp predicted')
# ax2.axis([x0,x1,y0,y1])
# plt.setp(ax2.get_yticklabels(), visible=False)

plt.style.use('fivethirtyeight')
# plt.annotate('Election Day 2016', size  = 11, xy=(-9, 350), xytext=(-9, 430),
#             arrowprops=dict(facecolor='black', shrink=0.05),
#             )
plt.legend(handles = [line1], loc = 1)
plt.tight_layout()
plt.show()

In [ ]:
x = np.linspace(y0,y1,100) 
p = skew_norm_pdf(x,mu,sd,alpha)

plt.plot(p,x)
plt.gca().invert_xaxis()
# ax0.text(-12,0.15,'mu = %.2f'%mu)
# ax0.text(-12,0.1,'alpha = %.2f'%alpha)
# ax0.text(-12,0.05,'sd = %.2f'%sd)
plt.show()

In [ ]:
dfobs = pd.read_csv('Output52.txt', sep = ' ', names = ['day','word','cnt','z'])
dfobs['lpobs'] = np.log(dfobs['cnt']/dfobs['z'])

dfobs= dfobs.sort_values('day', ascending = True)
#Create Bounds
n=63
mu = df.iloc[n,:]['mu']
sd = df.iloc[n,:]['sd']
alpha = df.iloc[n,:]['alpha']
drift = df.iloc[n,:]['drift']

y0,y1,x1,x0 = dfobs['lpobs'].min()-0.5, dfobs['lpobs'].max()+0.5, dfobs['day'].max(), dfobs['day'].min()
x = np.linspace(y0,y1,100) 
p = skew_norm_pdf(x,mu,sd,alpha)
d = skew_norm_pdf(x,mu,drift,0)
# plot it
from matplotlib import gridspec
gs = gridspec.GridSpec(3, 3)

bg_color = 'white'
fig = plt.figure(figsize=(15, 4), facecolor=bg_color) 
gs = gridspec.GridSpec(1, 2, width_ratios=[0.9, 6]) 
ax0 = plt.subplot(gs[0])
ax0.plot(p,x, linewidth = 1.2)
ax0.plot(d,x, linewidth = 1.2)
plt.gca().invert_xaxis()
ax0.text(0.35,-5,'mu = %.2f'%mu, size = 11)
ax0.text(0.35,-5.2,'alpha = %.2f'%alpha, size = 11)
ax0.text(0.35,-5.4,'sd = %.2f'%sd, size = 11)
ax0.set_title("Lp Distribution for: 'Trump'")
ax0.grid(False)
ax0.set_facecolor('white')
# plt.setp(ax0.get_yticklabels(), visible=False)

ax1 = plt.subplot(gs[1], sharey = ax0)
line1, = ax1.plot(dfobs['day'], dfobs['lpobs'], alpha = 0.7, linewidth = 1.2, label = 'lp observed')
ax1.axis([x0,x1,y0,y1])
ax1.set_ylabel("Log Probability")
ax1.set_xlabel('Day')
ax1.grid(False)
#####vertical line
xs = np.linspace(dfobs['lpobs'].min()-1, dfobs['lpobs'].max()+1,200)
vert_line_data = np.array([348 for i in range(len(xs))])
ax1.plot(vert_line_data, xs, 'r--', linewidth = 1.1) 
ax1.text(350,-8.5,'Election Day 2016', size = 14, fontweight = 'bold')
ax1.set_facecolor('white')
#horiz_line_data_2 = np.array([290 for i in range(len(xs))])
#ax1.plot(xs, horiz_line_data_2, 'k--', linewidth = 0.7) 
#ax1.text(-12,309,'Pipe Bomb in New York', size = 11, fontweight = 'bold')

# ax2 = ax1.twinx()
# line2, = ax2.plot(dftrumplp['mu'], dftrumplp['day'], '-.', alpha = 0.7, c = 'r', linewidth = 0.4, label = 'lp predicted')
# ax2.axis([x0,x1,y0,y1])
# plt.setp(ax2.get_yticklabels(), visible=False)

plt.style.use('fivethirtyeight')
plt.legend(handles = [line1], loc = 1, prop={'size': 14})
# plt.tight_layout(pad=0.1, w_pad=1, h_pad=0.1)
plt.show()

## Plot Clinton

In [ ]:
dfobs = pd.read_csv('Output207.txt', sep = ' ', names = ['day','word','cnt','z'])
dfobs['lpobs'] = np.log(dfobs['cnt']/dfobs['z'])

dfobs= dfobs.sort_values('day', ascending = True)
#Create Bounds
n=348
mu = df.iloc[n,:]['mu']
sd = df.iloc[n,:]['sd']
alpha = df.iloc[n,:]['alpha']
drift = df.iloc[n,:]['drift']

y0,y1,x1,x0 = dfobs['lpobs'].min()-1, dfobs['lpobs'].max()+0.5, dfobs['day'].max(), dfobs['day'].min()
x = np.linspace(y0,y1,100) 
p = skew_norm_pdf(x,mu,sd,alpha)
d = skew_norm_pdf(x,mu,drift,0)
# plot it
from matplotlib import gridspec
gs = gridspec.GridSpec(3, 3)


bg_color = 'white'
fig = plt.figure(figsize=(15, 4), facecolor=bg_color) 
gs = gridspec.GridSpec(1, 2, width_ratios=[0.9, 6]) 
ax0 = plt.subplot(gs[0])
ax0.plot(p,x, linewidth = 1.2)
ax0.plot(d,x, linewidth = 1.2)
plt.gca().invert_xaxis()
ax0.text(0.35,-6,'mu = %.2f'%mu, size = 11)
ax0.text(0.35,-6.3,'alpha = %.2f'%alpha, size = 11)
ax0.text(0.35,-6.6,'sd = %.2f'%sd, size = 11)
ax0.set_title("Lp Distribution for: 'Clinton'")
ax0.grid(False)
ax0.set_facecolor('white')
# plt.setp(ax0.get_yticklabels(), visible=False)

ax1 = plt.subplot(gs[1], sharey = ax0)
line1, = ax1.plot(dfobs['day'], dfobs['lpobs'], alpha = 0.7, linewidth = 1.2, label = 'lp observed')
ax1.axis([x0,x1,y0,y1])
ax1.set_ylabel("Log Probability")
ax1.set_xlabel('Day')
ax1.grid(False)
#####vertical line
xs = np.linspace(dfobs['lpobs'].min()-1, dfobs['lpobs'].max()+1,200)
vert_line_data = np.array([348 for i in range(len(xs))])
ax1.plot(vert_line_data, xs, 'r--', linewidth = 1.1) 
# ax1.text(350,-11,'Election Day 2016', size = 11, fontweight = 'bold')
ax1.set_facecolor('white')
#horiz_line_data_2 = np.array([290 for i in range(len(xs))])
#ax1.plot(xs, horiz_line_data_2, 'k--', linewidth = 0.7) 
#ax1.text(-12,309,'Pipe Bomb in New York', size = 11, fontweight = 'bold')

# ax2 = ax1.twinx()
# line2, = ax2.plot(dftrumplp['mu'], dftrumplp['day'], '-.', alpha = 0.7, c = 'r', linewidth = 0.4, label = 'lp predicted')
# ax2.axis([x0,x1,y0,y1])
# plt.setp(ax2.get_yticklabels(), visible=False)

plt.style.use('fivethirtyeight')
# plt.annotate('Election Day 2016', size  = 11, xy=(-9, 350), xytext=(-9, 430),
#             arrowprops=dict(facecolor='black', shrink=0.05),
#             )
plt.legend(handles = [line1], loc = 1)
# plt.tight_layout(pad=0.1, w_pad=1, h_pad=0.1)
plt.show()

## Obama Plot

In [ ]:
dfobs = pd.read_csv('Output267.txt', sep = ' ', names = ['day','word','cnt','z'])
dfobs['lpobs'] = np.log(dfobs['cnt']/dfobs['z'])

dfobs= dfobs.sort_values('day', ascending = True)
#Create Bounds
n=306
mu = df.iloc[n,:]['mu']
sd = df.iloc[n,:]['sd']
alpha = df.iloc[n,:]['alpha']
drift = df.iloc[n,:]['drift']

y0,y1,x1,x0 = dfobs['lpobs'].min()-1, dfobs['lpobs'].max()+0.5, dfobs['day'].max(), dfobs['day'].min()
x = np.linspace(y0,y1,100) 
p = skew_norm_pdf(x,mu,sd,alpha)
d = skew_norm_pdf(x,mu,drift,0)
# plot it
from matplotlib import gridspec
gs = gridspec.GridSpec(3, 3)

bg_color = 'white'
fig = plt.figure(figsize=(15, 4), facecolor=bg_color) 
gs = gridspec.GridSpec(1, 2, width_ratios=[0.9, 6]) 
ax0 = plt.subplot(gs[0])
ax0.plot(p,x, linewidth = 1.2)
ax0.plot(d,x, linewidth = 1.2)
plt.gca().invert_xaxis()
ax0.text(0.15,-6.5,'mu = %.2f'%mu, size = 11)
ax0.text(0.15,-6.7,'alpha = %.2f'%alpha, size = 11)
ax0.text(0.15,-6.9,'sd = %.2f'%sd, size = 11)
ax0.set_title("Lp Distribution for: 'Obama'")
ax0.grid(False)
ax0.set_facecolor('white')
# plt.setp(ax0.get_yticklabels(), visible=False)

ax1 = plt.subplot(gs[1], sharey = ax0)
line1, = ax1.plot(dfobs['day'], dfobs['lpobs'], alpha = 0.7, linewidth = 1.2, label = 'lp observed')
ax1.axis([x0,x1,y0,y1])
ax1.set_ylabel("Log Probability")
ax1.set_xlabel('Day')
ax1.grid(False)
#####vertical line
xs = np.linspace(dfobs['lpobs'].min()-1, dfobs['lpobs'].max()+1,200)
vert_line_data = np.array([348 for i in range(len(xs))])
ax1.plot(vert_line_data, xs, 'r--', linewidth = 1.1) 
ax1.set_facecolor('white')
# ax1.text(350,-10,'Election Day 2016', size = 11, fontweight = 'bold')
#horiz_line_data_2 = np.array([290 for i in range(len(xs))])
#ax1.plot(xs, horiz_line_data_2, 'k--', linewidth = 0.7) 
#ax1.text(-12,309,'Pipe Bomb in New York', size = 11, fontweight = 'bold')

# ax2 = ax1.twinx()
# line2, = ax2.plot(dftrumplp['mu'], dftrumplp['day'], '-.', alpha = 0.7, c = 'r', linewidth = 0.4, label = 'lp predicted')
# ax2.axis([x0,x1,y0,y1])
# plt.setp(ax2.get_yticklabels(), visible=False)

plt.style.use('fivethirtyeight')
# plt.annotate('Election Day 2016', size  = 11, xy=(-9, 350), xytext=(-9, 430),
#             arrowprops=dict(facecolor='black', shrink=0.05),
#             )
plt.legend(handles = [line1], loc = 1)
# plt.tight_layout(pad=0.1, w_pad=1, h_pad=0.1)
plt.show()

## Russia Plot

In [6]:
dfobs = pd.read_csv('Output470.txt', sep = ' ', names = ['day','word','cnt','z'])
dfobs['lpobs'] = np.log(dfobs['cnt']/dfobs['z'])

dfobs= dfobs.sort_values('day', ascending = True)

#Create Bounds
n=510
mu = df.iloc[n,:]['mu']
sd = df.iloc[n,:]['sd']
alpha = df.iloc[n,:]['alpha']
drift = df.iloc[n,:]['drift']

y0,y1,x1,x0 = dfobs['lpobs'].min()-0.5, dfobs['lpobs'].max()+0.5, dfobs['day'].max(), dfobs['day'].min()
x = np.linspace(y0,y1,100) 
p = skew_norm_pdf(x,mu,sd,alpha)
d = skew_norm_pdf(x,mu,drift,0)
# plot it
from matplotlib import gridspec
gs = gridspec.GridSpec(3, 3)

fig = plt.figure(figsize=(15, 4)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[0.9, 6]) 
ax0 = plt.subplot(gs[0])
ax0.plot(p,x, linewidth = 1.2)
ax0.plot(d,x, linewidth = 1.2)
plt.gca().invert_xaxis()
ax0.text(0.18,-7,'mu = %.2f'%mu, size = 11)
ax0.text(0.18,-7.2,'alpha = %.2f'%alpha, size = 11)
ax0.text(0.18,-7.4,'sd = %.2f'%sd, size = 11)
ax0.set_title("Lp Distribution for: 'Russia'")
ax0.grid(False)
# plt.setp(ax0.get_yticklabels(), visible=False)

ax1 = plt.subplot(gs[1], sharey = ax0)
line1, = ax1.plot(dfobs['day'], dfobs['lpobs'], alpha = 0.7, linewidth = 1.2, label = 'lp observed')
ax1.axis([x0,x1,y0,y1])
ax1.set_ylabel("Log Probability")
ax1.set_xlabel('Day')
ax1.grid(linewidth = 0.5)
#####vertical line
xs = np.linspace(dfobs['lpobs'].min()-1, dfobs['lpobs'].max()+1,200)
vert_line_data = np.array([407 for i in range(len(xs))])
ax1.plot(vert_line_data, xs, 'r--', linewidth = 0.7) 
ax1.text(410,-10,'ODNI Hacking Report Released ', size = 11, fontweight = 'bold')


plt.style.use('fivethirtyeight')
plt.legend(handles = [line1], loc = 1)
# plt.tight_layout(pad=0.1, w_pad=1, h_pad=0.1)
plt.show()

FileNotFoundError: [Errno 2] File b'Output470.txt' does not exist: b'Output470.txt'

In [2]:
dfobs = pd.read_csv('lpout8678.csv')
dfobs.head()

,day,lpobs,mu,low,high
0,0,-12.300727,-11.650798,-12.724914,-10.849594
1,1,-11.759449,-11.428539,-11.976183,-11.081318
2,2,-12.268341,-11.768647,-12.421712,-11.265469
3,3,-12.711091,-12.036927,-12.851723,-11.447490
4,4,-12.939349,-12.226702,-12.985067,-11.723431


In [ ]:
!jt -r